In [28]:
import pandas as pd
from classes.WindAPI import *
from sqlalchemy import *
from sqlalchemy.orm import Session
from create_ref_mariadb import mariadb_connection, windturbines, windfarms
from create_ref_mongodb import mongodb_connection
import pandas as pd
import plotly.express as px
from datetime import datetime, timedelta
import pytz

client_mongodb = mongodb_connection()
engine_mariadb = mariadb_connection()

In [29]:
def fetch_windturbines_data(engine):
    
    df = pd.read_sql(
    """
    SELECT wt.windturbine_id AS wtId, 
        wt.code AS wtCode,
        wf.windfarm_id AS wfId, 
        wf.code AS wfCode, 
        wf.latitude wfLat,
        wf.longitude wfLon
    FROM mariadb_itw.windturbines AS wt
    INNER JOIN mariadb_itw.windfarms AS wf
        ON wf.windfarm_id = wt.windfarm_id
    WHERE wt.last_scada_update IS NOT NULL
    ORDER BY wf.code ASC, wt.code ASC
    ;
    """,
        con=engine
    )
    

    return df

def fetch_windfarms_data(engine):
    
    df = pd.read_sql(
    """
    SELECT wf.windfarm_id AS wfId, 
        wf.code AS wfCode, 
        wf.latitude wfLat,
        wf.longitude wfLon
    FROM mariadb_itw.windfarms AS wf
    ORDER BY wf.code ASC;
    """,
        con=engine
    )
    

    return df

In [30]:


tz = pytz.timezone('Europe/Paris')
current_datetime = datetime.now(tz)
lastday_datetime = current_datetime - timedelta(hours=24)

scada_col = client_mongodb.scada
scada_data = scada_col.find({
    "log_date": {'$gte': lastday_datetime}
}).sort("log_date", 1)

dfScada = pd.DataFrame(list(scada_data))

dfTurbines = fetch_windturbines_data(engine_mariadb)

dfFinal = dfScada.merge(dfTurbines, left_on="windturbine_id", right_on="wtId")
dfFinal.head()


,_id,windturbine_id,wind_turbine,log_date,wind_speed_average,wind_direction,active_power,rotor_speed,external_ambient_temperature,wind_speed_estimated,...,gear_cooling_water_temp_avg,controller_top_temp_avg,amb_windspeed_windsensor1_avg,amb_windspeed_windsensor2_avg,wtId,wtCode,wfId,wfCode,wfLat,wfLon
0,648441e057b33cdd25a417f0,1ec6d7ce-93fa-6edc-a613-53533bbf8f7e,1ec6d7ce-93fa-6edc-a613-53533bbf8f7e,2023-06-09 12:50:00,5.2,77.6,266.3,9.5,28.0,4.9,...,NaN,39.0,4.8,4.6,1ec6d7ce-93fa-6edc-a613-53533bbf8f7e,E02,1ec6d7a1-4b96-67a4-9358-df2d2b033685,PEP,48.7218,4.46681
1,648441e057b33cdd25a417ef,1ec6d7ce-93fa-6edc-a613-53533bbf8f7e,1ec6d7ce-93fa-6edc-a613-53533bbf8f7e,2023-06-09 13:00:00,5.0,74.4,204.0,9.0,28.0,4.8,...,NaN,37.0,4.6,4.4,1ec6d7ce-93fa-6edc-a613-53533bbf8f7e,E02,1ec6d7a1-4b96-67a4-9358-df2d2b033685,PEP,48.7218,4.46681
2,648441e057b33cdd25a417ee,1ec6d7ce-93fa-6edc-a613-53533bbf8f7e,1ec6d7ce-93fa-6edc-a613-53533bbf8f7e,2023-06-09 13:10:00,4.4,75.6,96.3,8.7,28.0,4.1,...,NaN,36.0,3.7,3.7,1ec6d7ce-93fa-6edc-a613-53533bbf8f7e,E02,1ec6d7a1-4b96-67a4-9358-df2d2b033685,PEP,48.7218,4.46681
3,648441e057b33cdd25a417ed,1ec6d7ce-93fa-6edc-a613-53533bbf8f7e,1ec6d7ce-93fa-6edc-a613-53533bbf8f7e,2023-06-09 13:20:00,5.3,62.1,265.2,9.3,28.0,5.1,...,NaN,36.0,4.9,4.7,1ec6d7ce-93fa-6edc-a613-53533bbf8f7e,E02,1ec6d7a1-4b96-67a4-9358-df2d2b033685,PEP,48.7218,4.46681
4,648441e057b33cdd25a417ec,1ec6d7ce-93fa-6edc-a613-53533bbf8f7e,1ec6d7ce-93fa-6edc-a613-53533bbf8f7e,2023-06-09 13:30:00,4.6,72.9,109.9,9.0,28.0,4.1,...,NaN,36.0,4.0,3.9,1ec6d7ce-93fa-6edc-a613-53533bbf8f7e,E02,1ec6d7a1-4b96-67a4-9358-df2d2b033685,PEP,48.7218,4.46681


In [57]:
import plotly.express as px

for wfId in dfFinal['wfId'].unique():    
    wfCode = dfFinal[dfFinal['wfId'] == wfId]['wfCode'].unique()[0]
    fig = px.line(dfFinal[dfFinal['wfId'] == wfId], x="log_date", y="active_power", color='windturbine_id', title="Production parc {}".format(wfCode),
                  labels={'x': 'Production kW', 'y': 'Date'})
    fig.show()